In [1]:
# basic python data handling analysis modules
import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
sns.set()
import pickle, os, gc, re
# small utility functions
from utility import *

# interactive jupyter widgets!
# https://towardsdatascience.com/interactive-controls-for-jupyter-notebooks-f5c94829aee6
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

%matplotlib inline

In [2]:
# import warnings
# warnings.filterwarnings('ignore')
# # cleanliness!

In [3]:
import shutil
import tempfile
import urllib.request
import gzip
import datetime
import json
import time

Dataset_directory = ".."+os.sep+"Datasets"+os.sep
tmdb_dir = create_subdir(Dataset_directory, "themoviedb")

In [164]:
# import datetime
# datetime.datetime.now()

In [324]:
# https://developers.themoviedb.org/3/getting-started/daily-file-exports

# The export job runs every day starting at around 7:00 AM UTC, and all files are available by 8:00 AM UTC.

# http://files.tmdb.org/p/exports/movie_ids_04_28_2017.json.gz


# Here's something that would have been useful to know ahead of time:
# https://developers.themoviedb.org/3/getting-started/append-to-response

In [10]:
# import datetime

# utc_now = datetime.datetime.utcnow()

# if utc_now.hour<8: # upload for 'today' not necessary donw yet
#     utc_now = today - datetime.timedelta(days = 1)
    
# movie_ids_file = "http://files.tmdb.org/p/exports/movie_ids_"+utc_now.strftime("%m_%d_%Y")+".json.gz"
# movie_ids_file

In [16]:

utc_now = datetime.datetime.utcnow()

if utc_now.hour<8: # upload for 'today' not necessary donw yet
    utc_now = utc_now - datetime.timedelta(days = 1)
    
daily_file_export_list = ["movie","tv_series","person","collection","tv_network","keyword","production_company"]

for daily_export in daily_file_export_list:

    daily_file_export_file = "http://files.tmdb.org/p/exports/"+daily_export+"_ids_"+utc_now.strftime("%m_%d_%Y")+".json.gz"
    # movie_ids_file

    outFilePath = tmdb_dir + daily_file_export_file.split("/")[-1].replace(".gz","")

    with urllib.request.urlopen(daily_file_export_file) as response:
        with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
            shutil.copyfileobj(response, tmp_file)

    with gzip.open(tmp_file.name, mode='rt', encoding="utf-8") as f_in:
        with open(outFilePath, 'w', encoding="utf-8") as f_out:
            shutil.copyfileobj(f_in, f_out)

    daily_file_export_ids = pd.read_json(outFilePath, lines=True)
    daily_file_export_ids.describe(include='all')        

In [317]:
outFilePath = tmdb_dir + "movie_ids_06_29_2019.json"

movie_ids = pd.read_json(outFilePath, lines=True)
movie_ids.describe(include='all')

,adult,id,original_title,popularity,video
count,467797,467797.000000,467797,467797.000000,467797
unique,2,NaN,430051,NaN,2
top,False,NaN,Carmen,NaN,False
freq,443576,NaN,46,NaN,445325
mean,NaN,332134.697503,NaN,1.118390,NaN
std,NaN,175741.448301,NaN,1.995175,NaN
min,NaN,2.000000,NaN,0.000000,NaN
25%,NaN,191544.000000,NaN,0.600000,NaN
50%,NaN,347182.000000,NaN,0.600000,NaN
75%,NaN,482138.000000,NaN,0.841000,NaN


In [83]:
!pip install tmdbsimple

In [ ]:
# http://api.themoviedb.org/3/movie/16535?api_key=###

In [165]:
# import tmdbsimple as tmdb
text_file = open(tmdb_dir+"my_api_key.txt", "r")
# tmdb.API_KEY = text_file.readlines()[0]
api_key = text_file.readlines()[0]

In [98]:
# with urllib.request.urlopen("http://api.themoviedb.org/3/movie/16535?api_key="+tmdb.API_KEY) as response:
#     resp = response.readlines()
# #     with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
# #         shutil.copyfileobj(response, tmp_file)

In [155]:


tmdb_df = pd.DataFrame(columns = ['adult', 'backdrop_path', 'belongs_to_collection', 'budget', 'genres',
       'homepage', 'id', 'imdb_id', 'original_language', 'original_title',
       'overview', 'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'])

In [318]:
count = 0
fail_ids = []
start_timer = time.time()
for tmdb_id in movie_ids["id"][0:80].values:
    print(tmdb_id)
    try:
        with urllib.request.urlopen("http://api.themoviedb.org/3/movie/"+str(tmdb_id)+"?api_key="+api_key) as response:
            resp = response.readlines()[0]
        # just assuming it doesn't break!

        tmdb_df.loc[tmdb_id] = pd.DataFrame.from_dict(json.loads(resp), orient='index').T.values[0]
    except:
        print("failure at id = "+str(tmdb_id))
        fail_ids.append(tmdb_id)
        
    #X-RateLimit
    headers_dict = dict(response.headers.items())
    if 'X-RateLimit-Remaining' in headers_dict.keys():
        rate_limit_remaining = int(headers_dict['X-RateLimit-Remaining'])
        if rate_limit_remaining ==0:
            time.sleep(10)        
        
    count = count +1
    if (count % 40) == 0: # 40 api calls made
        finish_timer = time.time()
        seconds_passed = finish_timer-start_timer
        print("seconds_passed = ",seconds_passed)
        time.sleep( max(10 - seconds_passed,0) ) # respect the rate timer! no more than 40 requests every 10s
        
tmdb_df.to_msgpack( tmdb_dir+"tmdb_df"+".msgpack")

601
602
603
604
605
failure at id = 605
606
607
608
609
612
613
614
615
616
617
618
619
620
621
622
623
624
625
626
627
628
629
630
631
632
633
634
635
636
637
638
639
640
641
642
seconds_passed =  67.9778881072998
643
644
646
647
648
649
650
651
652
653
654
655
657
658
659
660
661
662
663
664
665
666
667
668
669
670
671
672
673
674
675
676
677
678
679
680
681
682
685
686
seconds_passed =  135.65075874328613


In [209]:
np.array_split(movie_ids["id"][0:90].values,3)

[array([601, 602, 603, 604, 605, 606, 607, 608, 609, 612, 613, 614, 615,
        616, 617, 618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628,
        629, 630, 631, 632], dtype=int64),
 array([633, 634, 635, 636, 637, 638, 639, 640, 641, 642, 643, 644, 646,
        647, 648, 649, 650, 651, 652, 653, 654, 655, 657, 658, 659, 660,
        661, 662, 663, 664], dtype=int64),
 array([665, 666, 667, 668, 669, 670, 671, 672, 673, 674, 675, 676, 677,
        678, 679, 680, 681, 682, 685, 686, 687, 688, 689, 690, 691, 692,
        693, 694, 695, 696], dtype=int64)]

In [215]:
int(np.ceil( len(data)/40 ))

3

In [224]:
batch = tmdb_id_list[ tmdb_id_batch*40:((tmdb_id_batch+1)*40)   ]
print(batch)
URLS = [ "http://api.themoviedb.org/3/movie/"+str(x)+"?api_key="+api_key for x in batch]
URLS

[687 688 689 690 691 692 693 694 695 696]


['http://api.themoviedb.org/3/movie/687?api_key=0c8e2eac3198de39966b29221e7e740b',
 'http://api.themoviedb.org/3/movie/688?api_key=0c8e2eac3198de39966b29221e7e740b',
 'http://api.themoviedb.org/3/movie/689?api_key=0c8e2eac3198de39966b29221e7e740b',
 'http://api.themoviedb.org/3/movie/690?api_key=0c8e2eac3198de39966b29221e7e740b',
 'http://api.themoviedb.org/3/movie/691?api_key=0c8e2eac3198de39966b29221e7e740b',
 'http://api.themoviedb.org/3/movie/692?api_key=0c8e2eac3198de39966b29221e7e740b',
 'http://api.themoviedb.org/3/movie/693?api_key=0c8e2eac3198de39966b29221e7e740b',
 'http://api.themoviedb.org/3/movie/694?api_key=0c8e2eac3198de39966b29221e7e740b',
 'http://api.themoviedb.org/3/movie/695?api_key=0c8e2eac3198de39966b29221e7e740b',
 'http://api.themoviedb.org/3/movie/696?api_key=0c8e2eac3198de39966b29221e7e740b']

In [312]:
outFilePath = movie_ids_06_29_2019


movie_ids = pd.read_json(outFilePath, lines=True)
movie_ids.describe(include='all')

,adult,id,original_title,popularity,video
count,467223,467223.000000,467223,467223.000000,467223
unique,2,NaN,429550,NaN,2
top,False,NaN,Carmen,NaN,False
freq,443083,NaN,46,NaN,444777
mean,NaN,331788.498929,NaN,1.111966,NaN
std,NaN,175573.963916,NaN,2.037446,NaN
min,NaN,2.000000,NaN,0.000000,NaN
25%,NaN,191249.000000,NaN,0.600000,NaN
50%,NaN,346870.000000,NaN,0.600000,NaN
75%,NaN,481636.500000,NaN,0.778000,NaN


In [319]:
import concurrent.futures
import urllib.request

# URLS = #[some list of urls]
from multiprocessing import Pool
# from multiprocessing.dummy import Pool as ThreadPool


# Retrieve a json string
def load_json_str(url, timeout=60):
    try:
        response = urllib.request.urlopen(url, timeout=timeout)
        output = response.readlines()[0]
    except Exception as exc:
        print('%r generated an exception: %s' % (url, exc))    
#         output = None
        output = None
    return output

# tmdb_id_list = movie_ids["id"][0:90].values
all_movie_ids = movie_ids["id"].values
remaining_movie_ids = list( set(all_movie_ids).difference(set(list(tmdb_df.index))) )

tmdb_id_list  = remaining_movie_ids
# start_timer = time.time()


# Make the Pool of workers
pool = ThreadPool(2)
# Open the urls in their own threads
# and return the results

#close the pool and wait for the work to finish



for tmdb_id_batch in range(0, np.ceil( len(tmdb_id_list)/40 ).astype('int') ):
    batch = tmdb_id_list[ tmdb_id_batch*40:((tmdb_id_batch+1)*40)   ]
#     print(batch)
    
    URLS = [ "http://api.themoviedb.org/3/movie/"+str(x)+"?api_key="+api_key for x in batch]
    
    start_timer = time.time()
    # We can use a with statement to ensure threads are cleaned up promptly
#     with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
#         # Start the load operations and mark each future with its URL
#         future_to_url = {executor.submit(load_json_str, url, 60): url for url in URLS}
#         for future in concurrent.futures.as_completed(future_to_url):
#             url = future_to_url[future]
#             try:
#                 data = future.result()
#                 tmdb_df.loc[tmdb_id] = pd.DataFrame.from_dict(json.loads(data), orient='index').T.values[0]
#                 # do json processing here
#             except Exception as exc:
#                 print('%r generated an exception: %s' % (url, exc))
# #             else:
# #                 print('%r page is %d bytes' % (url, len(data)))
                
    
    results = pool.map(load_json_str, URLS)
#     for res in results:
#         tmdb_df.loc[tmdb_id] = pd.DataFrame.from_dict(json.loads(res), orient='index').T.values[0]
    
#     tmdb_df.loc[batch] =         pd.DataFrame.from_dict( [json.loads(res) for res in results] ).values
    
    df = pd.DataFrame.from_dict( [json.loads(res) for res in results if res is not None] )
    df.index = [batch[res_no] for res_no in range(0,len(results)) if results[res_no] is not None]
    tmdb_df = tmdb_df.append(df)
    
    finish_timer = time.time()
    duration = finish_timer-start_timer
    print(duration)
    time.sleep( max( 0,12 - duration ) )

    
pool.close()
pool.join()    

'http://api.themoviedb.org/3/movie/594155?api_key=0c8e2eac3198de39966b29221e7e740b' generated an exception: HTTP Error 404: Not Found
6.092348337173462
6.588376760482788
6.5143725872039795
6.742385625839233
6.537374019622803
6.7593865394592285
6.5153725147247314
6.653380632400513
6.700383424758911
6.735385417938232
6.595377206802368
7.004400730133057
7.254414796829224
6.696382999420166
5.3223042488098145


In [363]:
# outFilePath =  tmdb_dir + "production_company_ids_06_30_2019.json"
outFilePath = tmdb_dir + "movie_ids_06_30_2019.json"
namespace = "movie"

movie_ids = pd.read_json(outFilePath, lines=True)
movie_ids.describe(include='all')

,adult,id,original_title,popularity,video
count,467940,467940.000000,467940,467940.000000,467940
unique,2,NaN,430179,NaN,2
top,False,NaN,Carmen,NaN,False
freq,443700,NaN,46,NaN,445460
mean,NaN,332220.303394,NaN,1.122532,NaN
std,NaN,175782.794324,NaN,2.049486,NaN
min,NaN,2.000000,NaN,0.000000,NaN
25%,NaN,191600.750000,NaN,0.600000,NaN
50%,NaN,347267.500000,NaN,0.600000,NaN
75%,NaN,482253.250000,NaN,0.841000,NaN


In [351]:
# movie_ids

In [17]:
# outFilePath =  tmdb_dir + "production_company_ids_06_30_2019.json"
outFilePath = tmdb_dir + "person_ids_07_09_2019.json"
namespace = "person"

movie_ids = pd.read_json(outFilePath, lines=True)
movie_ids.describe(include='all')

,adult,id,name,popularity
count,1412131,1.412131e+06,1412131,1.412131e+06
unique,2,NaN,1302749,NaN
top,False,NaN,Led Sarmiento,NaN
freq,1376768,NaN,54,NaN
mean,NaN,1.504165e+06,NaN,6.547549e-01
std,NaN,6.367168e+05,NaN,4.843602e-01
min,NaN,1.000000e+00,NaN,0.000000e+00
25%,NaN,1.235994e+06,NaN,6.000000e-01
50%,NaN,1.617222e+06,NaN,6.000000e-01
75%,NaN,1.990050e+06,NaN,6.000000e-01


In [5]:
tmdb_df = pd.DataFrame(columns = [])
secondary_tmdb_dir = "e:\\Datasets\\themoviedb\\"
# tmdb_df = pd.read_msgpack( secondary_tmdb_dir+"person_ids_df_as_of_08_07_2019"+".msgpack")
# too big!
tmdb_df_indices = pd.read_pickle( secondary_tmdb_dir+"person_ids_df_as_of_08_07_2019_indices"+".pickle")

In [6]:
text_file = open(tmdb_dir+"my_api_key.txt", "r")
api_key = text_file.readlines()[0]

In [12]:
# set( list(tmdb_df_indices.values) ).union( set(list(tmdb_df.index)) )

{2097152,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 2097164,
 2097165,
 2097160,
 18,
 19,
 20,
 2097171,
 22,
 23,
 24,
 2097168,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 2097184,
 42,
 43,
 44,
 45,
 2097190,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 2097200,
 2097201,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 2097215,
 73,
 74,
 75,
 2097219,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 2097227,
 85,
 86,
 2097230,
 2097231,
 2097232,
 90,
 2097234,
 92,
 93,
 2097246,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 2097250,
 108,
 109,
 110,
 2097261,
 112,
 113,
 114,
 2097262,
 116,
 117,
 118,
 2097270,
 120,
 2097264,
 122,
 123,
 2097267,
 2097268,
 126,
 2097271,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 2097286,
 138,
 139,
 140,
 141,
 142,
 2097287,
 2097288,
 2097289,
 2097290,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 2097301,
 159,
 16

In [18]:
import concurrent.futures
import urllib.request

# URLS = #[some list of urls]
from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool


# Retrieve a json string
def load_json_str(url, timeout=60):
    try:
        response = urllib.request.urlopen(url, timeout=timeout)
        output = response.readlines()[0]
    except Exception as exc:
        print('%r generated an exception: %s' % (url, exc))    
#         output = None
        output = None
    return output

# tmdb_id_list = movie_ids["id"][0:90].values
all_movie_ids = movie_ids["id"].values
# remove the ones in the current tmdb_df index
# remaining_movie_ids = list( set(all_movie_ids).difference(set(list(tmdb_df.index))) )

# remove the ones in the tmdb_df_indices file (i.e. not current tmdb_df)
# remaining_movie_ids = list( set(all_movie_ids).difference(set(list(tmdb_df_indices.values))) )


remaining_movie_ids = list( set(all_movie_ids).difference( set( list(tmdb_df_indices.values) ).union( set(list(tmdb_df.index)) ) ) ) 

tmdb_id_list  = remaining_movie_ids
# start_timer = time.time()


# Make the Pool of workers
pool = ThreadPool(3)
# Open the urls in their own threads
# and return the results

#close the pool and wait for the work to finish



for tmdb_id_batch in range(0, np.ceil( len(tmdb_id_list)/40 ).astype('int') ):
    batch = tmdb_id_list[ tmdb_id_batch*40:((tmdb_id_batch+1)*40)   ]
#     print(batch)
    #URLS = [ "http://api.themoviedb.org/3/"+namespace+"/"+str(x)+"?api_key="+api_key for x in batch]
#     URLS = [ "https://api.themoviedb.org/3/"+namespace+"/"+str(x)+"?api_key="+api_key+"&language=en-US&append_to_response=alternative_titles%2Ccredits%2Cexternal_ids%2Ckeywords%2Crelease_dates" for x in batch]
    URLS = [ "https://api.themoviedb.org/3/"+namespace+"/"+str(x)+"?api_key="+api_key+"&language=en-US&append_to_response=images%2Ccredits%2Cexternal_ids" for x in batch]    
# https://api.themoviedb.org/3/movie/687?api_key=0c8e2eac3198de39966b29221e7e740b&language=en-US&append_to_response=alternative_titles%2Ccredits%2Cexternal_ids%2Ckeywords%2Crelease_dates
    
    
    start_timer = time.time()
               
    
    results = pool.map(load_json_str, URLS)
    
    df = pd.DataFrame.from_dict( [json.loads(res) for res in results if res is not None] )
    df.index = [batch[res_no] for res_no in range(0,len(results)) if results[res_no] is not None]
    tmdb_df = tmdb_df.append(df)
    
    finish_timer = time.time()
    duration = finish_timer-start_timer
    print(duration)
    time.sleep( max( 0,11 - duration ) )

    
pool.close()
pool.join()    

9.412538528442383
9.321533203125
9.281530857086182
9.320533275604248
9.447540521621704
9.28153109550476
9.327533483505249
12.274701833724976
'https://api.themoviedb.org/3/person/2067666?api_key=0c8e2eac3198de39966b29221e7e740b&language=en-US&append_to_response=images%2Ccredits%2Cexternal_ids' generated an exception: HTTP Error 429: 
12.409709930419922
'https://api.themoviedb.org/3/person/2354459?api_key=0c8e2eac3198de39966b29221e7e740b&language=en-US&append_to_response=images%2Ccredits%2Cexternal_ids' generated an exception: HTTP Error 429: 
9.4175386428833
9.456541061401367
9.436539649963379
9.255529403686523
9.433539390563965
9.441540002822876
9.247529029846191
9.209526777267456
59.579407930374146
9.251529216766357
9.399537563323975
9.448540449142456
9.066518783569336
9.376536130905151
9.374536514282227
9.50554370880127
9.210526704788208
9.174524784088135
9.425539016723633
11.669667482376099
'https://api.themoviedb.org/3/person/2355265?api_key=0c8e2eac3198de39966b29221e7e740b&languag

In [ ]:
# tmdb_df.index.to_pickle( secondary_tmdb_dir+"person_ids_df_as_of_08_07_2019_indices"+".pickle")

In [21]:
# tmdb_df.reset_index()["index"].to_pickle( secondary_tmdb_dir+"person_ids_df_as_of_08_07_2019_indices"+".pickle")

In [10]:
tmdb_df.shape, len(remaining_movie_ids)

((370374, 17), 405599)

In [24]:
tmdb_df_indices.values

array([      1,       2,       3, ..., 1669705, 1669706, 1669707],
      dtype=int64)

In [25]:
tmdb_df.index

Int64Index([      1,       2,       3,       4,       5,       6,       7,
                  8,       9,      10,
            ...
            1669698, 1669699, 1669700, 1669701, 1669702, 1669703, 1669704,
            1669705, 1669706, 1669707],
           dtype='int64', length=1003261)

In [16]:
len(remaining_movie_ids)

729266

In [17]:
len(all_movie_ids)

1408860

In [9]:
gc.collect()

0

In [367]:
# load_json_str("https://api.themoviedb.org/3/movie/687?api_key=0c8e2eac3198de39966b29221e7e740b&language=en-US&append_to_response=alternative_titles%2Ccredits%2Cexternal_ids%2Ckeywords%2Crelease_dates")

In [368]:
# res = load_json_str("http://api.themoviedb.org/3/collection/645?api_key=0c8e2eac3198de39966b29221e7e740b")


In [258]:
# pd.DataFrame.from_dict( json.loads(res) )
# ~7 hrs -> 86,000 rows out of 450,000
# 36.4 -> another 30hrs
# 1/5 -> 300MB -> 1.5Gb (maybe save to other hd!)
# ram not an issue - 200MB for tmbd_df


def pretty_print(df):
    return display( HTML( df.to_html().replace("\\n","<br>") ) )

def sizeof_fmt(num, suffix='B'):
    ''' By Fred Cirera, after https://stackoverflow.com/a/1094933/1870254'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)


def memory_use(locs = locals().items()):
    gc.collect()
    # locals().items()
    for name, size in sorted(((name, sys.getsizeof(value)) for name,value in locs),
                             key= lambda x: -x[1])[:10]:
        print("{:>30}: {:>8}".format(name,sizeof_fmt(size)))

In [314]:
import sys
memory_use(locs = locals().items())

                      tmdb_df2:   1.4GiB
                   death_month:  61.5MiB
                    death_year:  61.5MiB
                          _134:  61.5MiB
                          _135:  61.5MiB
                          _288:  61.5MiB
                          _289:  61.5MiB
                          _237:  53.8MiB
                           _69:  52.1MiB
                        tuples:  52.1MiB


In [316]:
# import sys
# memory_use(locs = globals().items())

In [317]:
del death_month,death_year,_134,_135,_288,_289,_237,_69,tuples

In [315]:
gc.collect()

0

In [269]:
locs = globals().items()

In [20]:
# tmdb_df.to_msgpack( secondary_tmdb_dir+"person_ids_df_as_of_10_07_2019_final_part"+".msgpack")

In [21]:
tmdb_df.shape
# 310,000 out of 1,400,000
# 1/5-1/4
# 1.7Gb on disk, 400Mb in memory

# lot of memory leakage

(408923, 17)

In [23]:
tmdb_df2.shape

(1002234, 17)

In [313]:
tmdb_df2.dtypes

adult                            bool
also_known_as                  object
biography                      object
gender                          uint8
homepage                       object
id                             uint32
images                         object
imdb_id                       float64
known_for_department         category
name                           object
place_of_birth                 object
popularity                    float64
profile_path                   object
cast_tmdb_ids                  object
crew_tmdb_ids                  object
cast_credit_ids                object
crew_credit_ids                object
death_day                     float32
death_month                   float32
death_year                    float32
birth_day                     float32
birth_month                   float32
birth_year                    float32
external_id__facebook_id       object
external_id__freebase_id       object
external_id__freebase_mid      object
external_id_

In [284]:
tmdb_df2.dtypes[tmdb_df2.dtypes=="float64"].index

Index(['imdb_id', 'popularity', 'death_day', 'death_month', 'death_year',
       'birth_day', 'birth_month', 'birth_year', 'external_id__tvrage_id'],
      dtype='object')

In [311]:


tmdb_df2['death_day']   = tmdb_df2['death_day'].astype(np.float32)
tmdb_df2['death_month'] = tmdb_df2['death_month'].astype(np.float32)
tmdb_df2['death_year']  = tmdb_df2['death_year'].astype(np.float32)

tmdb_df2['birth_day']   = tmdb_df2['birth_day'].astype(np.float32)
tmdb_df2['birth_month'] = tmdb_df2['birth_month'].astype(np.float32)
tmdb_df2['birth_year']  = tmdb_df2['birth_year'].astype(np.float32)

tmdb_df2['external_id__tvrage_id'] = tmdb_df2['external_id__tvrage_id'].astype(np.float32)

In [312]:
tmdb_df2['gender'] = tmdb_df2['gender'].astype(np.uint8)
tmdb_df2['id'] = tmdb_df2['id'].astype(np.uint32)

In [310]:
(tmdb_df2['external_id__tvrage_id'].dropna() != tmdb_df2['external_id__tvrage_id'].dropna().astype(np.float32)).sum()

0

In [24]:
tmdb_df2 = tmdb_df2.append(tmdb_df)

In [319]:
tmdb_df2.shape

(1411157, 30)

In [321]:
tmdb_df3  = pd.read_msgpack( secondary_tmdb_dir+"person_ids_10_07_19_final"+".msgpack")

In [318]:
tmdb_df2.to_msgpack( secondary_tmdb_dir+"person_ids_10_07_19_final"+".msgpack")

In [322]:
tmdb_df3.columns

Index(['adult', 'also_known_as', 'biography', 'gender', 'homepage', 'id',
       'images', 'imdb_id', 'known_for_department', 'name', 'place_of_birth',
       'popularity', 'profile_path', 'cast_tmdb_ids', 'crew_tmdb_ids',
       'cast_credit_ids', 'crew_credit_ids', 'death_day', 'death_month',
       'death_year', 'birth_day', 'birth_month', 'birth_year',
       'external_id__facebook_id', 'external_id__freebase_id',
       'external_id__freebase_mid', 'external_id__imdb_id',
       'external_id__instagram_id', 'external_id__tvrage_id',
       'external_id__twitter_id'],
      dtype='object')

In [329]:
tmdb_df2[ tmdb_df2["also_known_as"]!=tmdb_df3["also_known_as"] ]

,adult,also_known_as,biography,gender,homepage,id,images,imdb_id,known_for_department,name,...,birth_day,birth_month,birth_year,external_id__facebook_id,external_id__freebase_id,external_id__freebase_mid,external_id__imdb_id,external_id__instagram_id,external_id__tvrage_id,external_id__twitter_id
1669708,False,[],,0,None,1669708,{'profiles': []},NaN,Acting,Maurizio Vanadia,...,NaN,NaN,NaN,None,None,None,None,None,NaN,None
1669709,False,[],,0,None,1669709,{'profiles': []},NaN,Acting,Mathieu Massuard,...,NaN,NaN,NaN,None,None,None,None,None,NaN,None
1669710,False,[],,0,None,1669710,{'profiles': []},NaN,Directing,Mathieu Massuard,...,NaN,NaN,NaN,None,None,None,None,None,NaN,None
1669711,False,[],,0,None,1669711,{'profiles': []},NaN,Acting,Mathieu Massuard,...,NaN,NaN,NaN,None,None,None,None,None,NaN,None
1669712,False,[],,0,None,1669712,{'profiles': []},NaN,Acting,Mathieu Massuard,...,NaN,NaN,NaN,None,None,None,None,None,NaN,None
1669713,False,[],,0,None,1669713,{'profiles': []},NaN,Directing,Alexandre Brisou,...,NaN,NaN,NaN,None,None,None,None,None,NaN,None
1669714,False,[],,0,None,1669714,{'profiles': []},NaN,Acting,Alexandre Brisou,...,NaN,NaN,NaN,None,None,None,None,None,NaN,None
1669715,False,[],,0,None,1669715,{'profiles': []},NaN,Acting,Isabel Seabra,...,NaN,NaN,NaN,None,None,None,None,None,NaN,None
1669716,False,[],,0,None,1669716,{'profiles': []},NaN,Acting,Yvon Arai,...,NaN,NaN,NaN,None,None,None,None,None,NaN,None
1669717,False,[],,0,None,1669717,{'profiles': []},NaN,Writing,Rupali Guha,...,NaN,NaN,NaN,None,None,None,None,None,NaN,None


In [330]:
tmdb_df3[ tmdb_df2["also_known_as"]!=tmdb_df3["also_known_as"] ]

,adult,also_known_as,biography,gender,homepage,id,images,imdb_id,known_for_department,name,...,birth_day,birth_month,birth_year,external_id__facebook_id,external_id__freebase_id,external_id__freebase_mid,external_id__imdb_id,external_id__instagram_id,external_id__tvrage_id,external_id__twitter_id
1669708,False,(),,0,None,1669708,{'profiles': ()},NaN,Acting,Maurizio Vanadia,...,NaN,NaN,NaN,None,None,None,None,None,NaN,None
1669709,False,(),,0,None,1669709,{'profiles': ()},NaN,Acting,Mathieu Massuard,...,NaN,NaN,NaN,None,None,None,None,None,NaN,None
1669710,False,(),,0,None,1669710,{'profiles': ()},NaN,Directing,Mathieu Massuard,...,NaN,NaN,NaN,None,None,None,None,None,NaN,None
1669711,False,(),,0,None,1669711,{'profiles': ()},NaN,Acting,Mathieu Massuard,...,NaN,NaN,NaN,None,None,None,None,None,NaN,None
1669712,False,(),,0,None,1669712,{'profiles': ()},NaN,Acting,Mathieu Massuard,...,NaN,NaN,NaN,None,None,None,None,None,NaN,None
1669713,False,(),,0,None,1669713,{'profiles': ()},NaN,Directing,Alexandre Brisou,...,NaN,NaN,NaN,None,None,None,None,None,NaN,None
1669714,False,(),,0,None,1669714,{'profiles': ()},NaN,Acting,Alexandre Brisou,...,NaN,NaN,NaN,None,None,None,None,None,NaN,None
1669715,False,(),,0,None,1669715,{'profiles': ()},NaN,Acting,Isabel Seabra,...,NaN,NaN,NaN,None,None,None,None,None,NaN,None
1669716,False,(),,0,None,1669716,{'profiles': ()},NaN,Acting,Yvon Arai,...,NaN,NaN,NaN,None,None,None,None,None,NaN,None
1669717,False,(),,0,None,1669717,{'profiles': ()},NaN,Writing,Rupali Guha,...,NaN,NaN,NaN,None,None,None,None,None,NaN,None


In [331]:
tmdb_df2.loc[1669720,"also_known_as"], tmdb_df3.loc[1669720,"also_known_as"], 

(['Sutha Raghunathan', 'Sudha Ragunathan', 'Sutha Ragunathan'],
 ('Sutha Raghunathan', 'Sudha Ragunathan', 'Sutha Ragunathan'))

In [332]:
tmdb_df3.shape, tmdb_df2.shape

((1411157, 30), (1411157, 30))

In [62]:
cast_tmdb_ids = tmdb_df2["credits"].apply(lambda x: [y["id"] for y in x["cast"]] if x else np.nan)
crew_tmdb_ids = tmdb_df2["credits"].apply(lambda x: [y["id"] for y in x["crew"]] if x else np.nan)

cast_credit_ids = tmdb_df2["credits"].apply(lambda x: [y["credit_id"] for y in x["cast"]] if x else np.nan)
crew_credit_ids = tmdb_df2["credits"].apply(lambda x: [y["credit_id"] for y in x["crew"]] if x else np.nan)

In [64]:
tmdb_df2["cast_tmdb_ids"] = cast_tmdb_ids
tmdb_df2["crew_tmdb_ids"] = crew_tmdb_ids
tmdb_df2["cast_credit_ids"] = cast_credit_ids
tmdb_df2["crew_credit_ids"] = crew_credit_ids

In [65]:
tmdb_df2.drop("credits",axis=1,inplace=True)

In [70]:
tmdb_df2["known_for_department"] = tmdb_df2["known_for_department"].astype('category')

In [92]:
tmdb_df2["imdb_id"].replace("", np.nan,inplace=True)
tmdb_df2["imdb_id"] = tmdb_df2["imdb_id"].apply(lambda x: x.replace("/", "") if pd.notnull(x) else np.nan)

# tmdb_df2["imdb_id"] = 
tmdb_df2["imdb_id"] = tmdb_df2["imdb_id"].apply(lambda x: int(x[2:]) if pd.notnull(x) else np.nan)

In [131]:
9030/birth_month.shape[0]
# .6%

0.006399004504814135

9030

In [168]:
# we can't use to_datetime because we have dates outside the standard timestampe range (authors born <17th century)
birth_dates = tmdb_df2["birthday"].replace("2018-05-01",np.nan).replace("",np.nan)
birth_day   = birth_dates.apply(lambda x: int(x[8:10]) if pd.notnull(x) else np.nan)
birth_month = birth_dates.apply(lambda x: int(x[5:7]) if pd.notnull(x) else np.nan)
birth_year  = birth_dates.apply(lambda x: int(x[0:4]) if pd.notnull(x) else np.nan)

placeholder_month_day = ( (birth_month==1)&(birth_day==1) )
birth_day[placeholder_month_day] = np.nan
birth_month[placeholder_month_day] = np.nan
# placeholder_month_day = birth_dates.applymap(lambda x: (x.day==1) & (x.month==1) if pd.notnull(x) else np.nan)

# tmdb_df2.loc[ tmdb_df2["deathday"].dropna()[tmdb_df2["deathday"].dropna().apply(lambda x: False if re.match("\d{4}-\d{2}-\d{2}",x) else True)].index , "deathday"]
# # bad formats ahoy!

death_dates = tmdb_df2["deathday"].replace("2018-05-01",np.nan).replace("",np.nan)
death_dates.loc[14358] = "1933-09-01"
death_dates.loc[46872] = "1856-07-29"
death_dates.loc[49433] = "2017-05-03"
death_dates.loc[109716] = np.nan
death_dates.loc[568848] = "2014-02-21"
death_dates.loc[1037222] = "2005-02-21"
death_dates.loc[1062049] = "1961-01-01"
death_dates.loc[1180849] = "2002-12-10"
death_dates.loc[1223553] = "1931-12-01"
death_dates.loc[1275241] = "2004-08-01"
death_dates.loc[1298986] = "2006-01-20"
death_dates.loc[1749144] = "1980-09-07"
death_dates.loc[1921687] = "2016-01-16"
death_dates.loc[1970555] = "1994-01-30"
death_dates.loc[2021204] = "2017-05-01"
death_dates.loc[2028131] = "1987-01-11"
death_dates.loc[1139622] = "2004-01-01"

death_day   = death_dates.apply(lambda x: int(x[8:10]) if pd.notnull(x) else np.nan)
death_month = death_dates.apply(lambda x: int(x[5:7]) if pd.notnull(x) else np.nan)
death_year  = death_dates.apply(lambda x: int(x[0:4]) if pd.notnull(x) else np.nan)

# deaths in the future deeemed automatically illegitimate!
future_deaths = death_year[death_year>=2020].index
death_day.loc[future_deaths] = np.nan
death_month.loc[future_deaths] = np.nan
death_year.loc[future_deaths] = np.nan

death_day[(death_day<1)|(death_day>31)] = np.nan
death_month[(death_month<1)|(death_month>12)] = np.nan

placeholder_month_day = ( (death_month==1)&(death_day==1) )
death_day[placeholder_month_day] = np.nan
death_month[placeholder_month_day] = np.nan


In [169]:
tmdb_df2["death_day"] = death_day
tmdb_df2["death_month"] = death_month
tmdb_df2["death_year"] = death_year

tmdb_df2["birth_day"] = birth_day
tmdb_df2["birth_month"] = birth_month
tmdb_df2["birth_year"] = birth_year


In [171]:
tmdb_df2.drop(["birthday","deathday"],axis=1,inplace=True)

In [175]:
flatten = lambda l: [item for sublist in l for item in sublist]

In [204]:
# tmdb_df2[ tmdb_df2["external_ids"].apply(lambda x: isinstance(x,tuple)) ]

,adult,also_known_as,biography,external_ids,gender,homepage,id,images,imdb_id,known_for_department,...,cast_tmdb_ids,crew_tmdb_ids,cast_credit_ids,crew_credit_ids,death_day,death_month,death_year,birth_day,birth_month,birth_year
583098,False,(),,(),1,None,583098,{'profiles': ()},607042.0,Editing,...,[],"[78086, 308193, 881]",[],"[52fe498cc3a368484e1310e9, 54809878c3a36829b20...",NaN,NaN,NaN,NaN,NaN,NaN
1415395,False,(),,(),0,None,1415395,{'profiles': ()},NaN,Directing,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [257]:
# pd.DataFrame.from_dict( tmdb_df2["external_ids"].drop([583098,1415395]).values )

In [252]:
df = pd.DataFrame( list( tmdb_df2["external_ids"].drop([583098,1415395]).values ),
             index = tmdb_df2["external_ids"].drop([583098,1415395]).index )
df.drop(["cast","crew","id"],axis=1,inplace=True)
df.loc[583098] = [None, None, None, None, None, np.nan, None]
df.loc[1415395] = [None, None, None, None, None, np.nan, None]
df.loc[1] = [None,'/en/george_lucas','/m/0343h','nm0000184',None,6490,""]

df.columns = "external_id__"+df.columns
tmdb_df2[df.columns] = df

In [255]:
tmdb_df2.drop("external_ids",axis=1,inplace=True)

In [248]:
tmdb_df2.shape

(1411157, 24)

In [254]:
tmdb_df2.columns

Index(['adult', 'also_known_as', 'biography', 'external_ids', 'gender',
       'homepage', 'id', 'images', 'imdb_id', 'known_for_department', 'name',
       'place_of_birth', 'popularity', 'profile_path', 'cast_tmdb_ids',
       'crew_tmdb_ids', 'cast_credit_ids', 'crew_credit_ids', 'death_day',
       'death_month', 'death_year', 'birth_day', 'birth_month', 'birth_year',
       'external_id__facebook_id', 'external_id__freebase_id',
       'external_id__freebase_mid', 'external_id__imdb_id',
       'external_id__instagram_id', 'external_id__tvrage_id',
       'external_id__twitter_id'],
      dtype='object')

In [256]:
tmdb_df2.dtypes

adult                            bool
also_known_as                  object
biography                      object
gender                          int64
homepage                       object
id                              int64
images                         object
imdb_id                       float64
known_for_department         category
name                           object
place_of_birth                 object
popularity                    float64
profile_path                   object
cast_tmdb_ids                  object
crew_tmdb_ids                  object
cast_credit_ids                object
crew_credit_ids                object
death_day                     float64
death_month                   float64
death_year                    float64
birth_day                     float64
birth_month                   float64
birth_year                    float64
external_id__facebook_id       object
external_id__freebase_id       object
external_id__freebase_mid      object
external_id_

In [134]:
tmdb_df2["imdb_id"]#.apply(lambda x: int(x[2:]) if pd.notnull(x) else np.nan)

1              184.0
2              434.0
3              148.0
4              402.0
5             1088.0
6              355.0
7             4056.0
8           881279.0
9           910237.0
10          677037.0
11          721675.0
12         1071252.0
13             983.0
14            1122.0
18            4951.0
19            5049.0
20            1610.0
22          402032.0
23          402730.0
24             709.0
26          343165.0
27          744839.0
28          277704.0
29            5494.0
30          823330.0
31             158.0
32             705.0
33             641.0
34          932112.0
35             398.0
             ...    
2071676          NaN
2088079          NaN
2088083    3111311.0
2088087          NaN
2357514          NaN
2096443          NaN
2080094          NaN
2357540          NaN
2354297          NaN
2354298          NaN
2357229          NaN
2072091          NaN
2072092          NaN
2072095    1597655.0
2072096      31527.0
2072099          NaN
2072100      

In [274]:
tmdb_df2.dtypes

adult                            bool
also_known_as                  object
biography                      object
gender                          int64
homepage                       object
id                              int64
images                         object
imdb_id                       float64
known_for_department         category
name                           object
place_of_birth                 object
popularity                    float64
profile_path                   object
cast_tmdb_ids                  object
crew_tmdb_ids                  object
cast_credit_ids                object
crew_credit_ids                object
death_day                     float64
death_month                   float64
death_year                    float64
birth_day                     float64
birth_month                   float64
birth_year                    float64
external_id__facebook_id       object
external_id__freebase_id       object
external_id__freebase_mid      object
external_id_

In [327]:
# (1400000/4)/3600
# getting all people data -> 97hrs!

In [22]:
len(all_movie_ids)

1408860

In [23]:
tmdb_df.shape[0]

1002234

In [36]:
tmdb_dir

'..\\Datasets\\\\themoviedb\\'

In [328]:
# #X-RateLimit
# headers_dict = dict(response.headers.items())
# if 'X-RateLimit-Remaining' in headers_dict.keys():
#     rate_limit_remaining = int(headers_dict['X-RateLimit-Remaining'])
#     if rate_limit_remaining ==0:
#         time.sleep(10)


In [329]:
# headers_dict = dict(response.headers.items())
# headers_dict

In [321]:
# tmdb_df.to_msgpack( tmdb_dir+"tmdb_df_final"+".msgpack")

In [331]:
# ((467223/40)*10) / 3600

In [330]:
# 1200 -> ~1MB
# 467223/1200
# -> 400MB

In [ ]:
# 32 hours starting ~ 17ish Wednesday -> 1am Friday morning ... some fixes later, done on Sunday!